## Big Data Day 2 Afternoon Assignment

In this assignment, we will learn about machine learning with Dask. We will use the market basket dataset loaded below and cluster our data.

In [1]:
# !pip install dask-ml --quiet
# !pip install -U ipykernel --quiet
# !pip install aiohttp
# !pip install dask distributed --upgrade
# !pip install scikit-learn==0.23.2

In [2]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

In [3]:
!pip list scikit-learn

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
aiohttp                       3.7.3          
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.1          
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
async-timeout                 3.0.1          
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.3.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.9.0          
backcall                      0.2.0          
beautifulsoup4                4.6.

In [4]:
market_basket = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Big%20Data/Mall_Customers.csv')

In [5]:
market_basket.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


To cluster our data, we must first examine and process it. The first step is to check for missing data. Do this in the cell below. If there is missing data, drop all rows containing missing data.

In [6]:
# Answer below:

market_basket.isnull().sum().compute()

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

Next, we will get rid of the customer ID column. Drop the column and assign the resulting dataframe to a new variable.

In [7]:
# Answer below:

market_basket = market_basket.drop(columns=['CustomerID'])
market_basket.head()

,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40


We will now create a dummy variable from the gender variable.

In [8]:
# Answer below:

market_basket.Gender = market_basket.Gender.map({'Male':1, 'Female':0})
market_basket.head()

,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,19,15,39
1,1,21,15,81
2,0,20,16,6
3,0,23,16,77
4,0,31,17,40


Since the data has different scales, we will scale all columns using min max scaling. We will write our own min max scaling function since the minmax scaler will produce numpy arrays instead of dask arrays. Recall that min max scaling requires finding the min and the max. We subtract the min from each observation and divide by the difference between the max and the min. Complete the function below.

In [9]:
# Answer below:

def dask_min_max(x):
    #Input: dask column
    #Output: dask column that has been min max scaled
    
    x_min = x.min()
    x_max = x.max()
    return (x - x_min)/ (x_max - x_min)

dask_min_max(market_basket.Age).compute()


0      0.019231
1      0.057692
2      0.038462
3      0.096154
4      0.250000
         ...   
195    0.326923
196    0.519231
197    0.269231
198    0.269231
199    0.230769
Name: Age, Length: 200, dtype: float64

Transform all columns using the scaler you wrote above.

In [10]:
# Answer below:

market_basket_scaled = dask_min_max(market_basket)
market_basket_scaled.head()


,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1.0,0.019231,0.000000,0.387755
1,1.0,0.057692,0.000000,0.816327
2,0.0,0.038462,0.008197,0.051020
3,0.0,0.096154,0.008197,0.775510
4,0.0,0.250000,0.016393,0.397959


Import the dask clustering function and cluster the data. Use 3 clusters and print the centroids.

In [11]:
# Answer below

from dask_ml.cluster import KMeans

In [13]:
kmeans = KMeans(n_clusters=3)

In [14]:
kmeans.fit(market_basket_scaled)

KMeans(n_clusters=3)

In [15]:
kmeans.cluster_centers_

array([[0.        , 0.38650412, 0.36270492, 0.51557945],
       [1.        , 0.60456731, 0.3886612 , 0.28784014],
       [1.        , 0.19711538, 0.3852459 , 0.72117347]])